# Task: Bert Model을 사용하여 BBC 뉴스 기사의 category를 분류

# Load data & Import packages

In [ ]:
%%capture
!pip install transformers

In [ ]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

In [ ]:
df = pd.read_csv('bbc-text.csv')

In [ ]:
df.head(10)

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
5,politics,howard hits back at mongrel jibe michael howar...
6,politics,blair prepares to name poll date tony blair is...
7,sport,henman hopes ended in dubai third seed tim hen...
8,sport,wilkinson fit to face edinburgh england captai...
9,entertainment,last star wars not for children the sixth an...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   text      2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [ ]:
df.groupby('category').count()

,text
category,
business,510
entertainment,386
politics,417
sport,511
tech,401


# BertTokenizer

토크나이저로 pretrain된 BERT의 BertTokenizer를 갖고 옵니다.

- bert-base-uncased : 108MB param, all lowercase
- bert-large-cased : 340MB param, both upper and lower
- bert-base-cased : 108MB param, multi language, both upper and lower


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'business':0,
          'entertainment':1,
          'sport':2,
          'tech':3,
          'politics':4
          }

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

# Dataset

In [ ]:
class Dataset(torch.utils.data.Dataset):

    # 클래스 초기화 메서드
    def __init__(self, df):
        # 데이터프레임 df의 'category' 컬럼 값을 labels 딕셔너리를 통해 숫자로 변환하여 self.labels에 저장
        self.labels = [labels[label] for label in df['category']]
        # 데이터프레임 df의 'text' 컬럼 값을 토크나이저를 사용하여 토큰화하고 텐서로 변환하여 self.texts에 저장
        self.texts = [tokenizer(text,
                               padding='max_length', max_length=512, truncation=True,
                               return_tensors="pt") for text in df['text']]

    # 클래스의 레이블 목록을 반환하는 메서드
    def classes(self):
        return self.labels

    # 데이터셋의 길이를 반환하는 메서드
    def __len__(self):
        return len(self.labels)

    # 특정 인덱스의 배치 레이블을 반환하는 메서드
    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    # 특정 인덱스의 배치 텍스트를 반환하는 메서드
    def get_batch_texts(self, idx):
        return self.texts[idx]

    # 특정 인덱스의 텍스트와 레이블을 반환하는 메서드
    def __getitem__(self, idx):
        # 해당 인덱스의 배치 텍스트를 가져옴
        batch_texts = self.get_batch_texts(idx)
        # 해당 인덱스의 배치 레이블을 가져옴
        batch_y = self.get_batch_labels(idx)

        # 배치 텍스트와 배치 레이블을 튜플로 반환
        return batch_texts, batch_y


# Train & Evaluate BertClassifier

pretrain된 BertModel을 불러옵니다. 다른 간단한 층들도 같이 쌓아줍니다.

- bert-base-cased: 12-layer, 768-hidden, 12-self attention heads, 110M parameters. Trained on cased English text.


다른 종류들의 pretrianed model은 아래 링크에서 확인할 수 있습니다.

https://huggingface.co/transformers/v2.9.1/pretrained_models.html

In [ ]:
# PyTorch의 nn.Module을 상속받아 BERT 기반 분류기 클래스를 정의
class BertClassifier(nn.Module):

    # 클래스 초기화 메서드
    def __init__(self, dropout=0.5):
        # 부모 클래스의 초기화 메서드를 호출
        super(BertClassifier, self).__init__()

        # 미리 학습된 BERT 모델을 로드
        self.bert = BertModel.from_pretrained('bert-base-cased')
        # 드롭아웃 레이어를 설정, 드롭아웃 비율은 기본값 0.5
        self.dropout = nn.Dropout(dropout)
        # 선형 레이어를 설정, 입력 차원 768(BERT의 출력 차원)에서 출력 차원 5(분류 클래스 수)
        self.linear = nn.Linear(768, 5)
        # ReLU 활성화 함수 설정
        self.relu = nn.ReLU()

    # 순전파 메서드
    def forward(self, input_id, mask):
        # BERT 모델에 입력 아이디와 어텐션 마스크를 전달하여 출력 획득
        _, pooled_output = self.bert(input_ids=input_id, attention_mask=mask, return_dict=False)
        # 드롭아웃 레이어를 통과시켜 과적합 방지
        dropout_output = self.dropout(pooled_output)
        # 선형 레이어를 통과시켜 예측값 생성
        linear_output = self.linear(dropout_output)
        # ReLU 활성화 함수를 적용하여 비선형성 추가
        final_layer = self.relu(linear_output)

        # 최종 출력값 반환
        return final_layer


In [ ]:
def train(model, train_data, val_data, learning_rate, epochs):
    # 학습 데이터와 검증 데이터를 Dataset 클래스를 이용해 처리
    train, val = Dataset(train_data), Dataset(val_data)

    # 학습 데이터로더와 검증 데이터로더 생성, 배치 크기는 2로 설정
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    # CUDA 사용 가능 여부 확인 및 장치 설정
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # 손실 함수와 옵티마이저 설정
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    # CUDA 사용 시 모델과 손실 함수를 GPU로 이동
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    # 지정된 에포크 수만큼 학습 반복
    for epoch_num in range(epochs):
        # 학습 데이터에 대한 정확도와 손실 초기화
        total_acc_train = 0
        total_loss_train = 0

        # 학습 데이터 배치 처리
        for train_input, train_label in tqdm(train_dataloader):
            # 학습 레이블과 입력 데이터의 어텐션 마스크, 입력 아이디를 GPU로 이동
            train_label = train_label.to(device)
            mask = train_input['attention_mask'].squeeze(1).to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            # 입력 데이터 형상 확인 및 출력
            print(f'input_id shape: {input_id.shape}')
            print(f'mask shape: {mask.shape}')

            # 모델에 입력 데이터를 전달하여 출력값 획득
            output = model(input_id, mask)

            # 손실 함수 계산
            batch_loss = criterion(output, train_label.long())
            total_loss_train += batch_loss.item()

            # 정확도 계산
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            # 역전파 단계와 옵티마이저 업데이트
            model.zero_grad()
            batch_loss.backward()
            optimizer.step()

        # 검증 데이터에 대한 정확도와 손실 초기화
        total_acc_val = 0
        total_loss_val = 0

        # 검증 데이터 배치 처리, 그래디언트 계산 중지
        with torch.no_grad():
            for val_input, val_label in val_dataloader:
                # 검증 레이블과 입력 데이터의 어텐션 마스크, 입력 아이디를 GPU로 이동
                val_label = val_label.to(device)
                mask = val_input['attention_mask'].squeeze(1).to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                # 입력 데이터 형상 확인 및 출력
                print(f'input_id shape: {input_id.shape}')
                print(f'mask shape: {mask.shape}')

                # 모델에 입력 데이터를 전달하여 출력값 획득
                output = model(input_id, mask)

                # 손실 함수 계산
                batch_loss = criterion(output, val_label.long())
                total_loss_val += batch_loss.item()

                # 정확도 계산
                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc

        # 학습 및 검증 결과 출력
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | '
            f'Train Accuracy: {total_acc_train / len(train_data): .3f} | '
            f'Val Loss: {total_loss_val / len(val_data): .3f} | '
            f'Val Accuracy: {total_acc_val / len(val_data): .3f}'
        )


In [ ]:
# 모델을 평가하는 함수 정의
def evaluate(model, test_data):
    # 테스트 데이터를 Dataset 클래스를 이용해 처리
    test = Dataset(test_data)

    # 테스트 데이터로더 생성, 배치 크기는 2로 설정
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    # CUDA 사용 가능 여부 확인 및 장치 설정
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # CUDA 사용 시 모델을 GPU로 이동
    if use_cuda:
        model = model.cuda()

    # 테스트 데이터에 대한 정확도 초기화
    total_acc_test = 0

    # 그래디언트 계산 중지
    with torch.no_grad():
        # 테스트 데이터 배치 처리
        for test_input, test_label in test_dataloader:
            # 테스트 레이블과 입력 데이터의 어텐션 마스크, 입력 아이디를 GPU로 이동
            test_label = test_label.to(device)
            mask = test_input['attention_mask'].squeeze(1).to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            # 입력 데이터 형상 확인 및 출력
            print(f'input_id shape: {input_id.shape}')
            print(f'mask shape: {mask.shape}')

            # 모델에 입력 데이터를 전달하여 출력값 획득
            output = model(input_id, mask)

            # 정확도 계산
            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc

    # 테스트 결과 출력
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')


In [ ]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

1780 222 223


mask shape 관련 에러가 발생하여 mask의 형상을 squeeze(1)을 사용해 [batch_size, sequence_length]로 맞춰 BertModel이 기대하는 형식과 일치시켜 해결

In [ ]:
EPOCHS = 2 #EPOCH 수 늘려보기!
model = BertClassifier()
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)

  0%|          | 0/890 [00:00<?, ?it/s]

input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])


  0%|          | 3/890 [00:01<04:14,  3.48it/s]

input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])


  1%|          | 5/890 [00:01<02:44,  5.38it/s]

...

100%|█████████▉| 888/890 [01:41<00:00,  8.68it/s]

input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])


100%|██████████| 890/890 [01:42<00:00,  8.72it/s]

input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])


input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torc

In [ ]:
evaluate(model, df_test)

input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torc

In [ ]:
EPOCHS = 5 #EPOCH 5로 늘려서 진행
model = BertClassifier()
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)

  0%|          | 2/890 [00:00<01:08, 12.95it/s]

input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])


  0%|          | 4/890 [00:00<01:27, 10.14it/s]

input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])


  1%|          | 6/890 [00:00<01:32,  9.53it/s]

...

100%|█████████▉| 889/890 [01:44<00:00,  8.60it/s]

input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])


100%|██████████| 890/890 [01:44<00:00,  8.54it/s]

input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])


input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torc

In [ ]:
evaluate(model, df_test)

input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torch.Size([2, 512])
mask shape: torch.Size([2, 512])
input_id shape: torc

Epoch를 2 -> 5로 늘려서 진행한 결과 Accuracy가 0.857 -> 0.991로 유의미하게 증가